In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from utils import loader
from utils import debugger 
from utils import preprocessing as pp 
from utils import misc
from utils import constant
import implementations as impl
import cost
import model

In [3]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = loader.load_csv_data(DATA_TRAIN_PATH)

In [5]:
to_replace = [(constant.UNDEF_VAL, 'most_frequent')]

augment_param = {
    'degrees': [2],
    'add_bias' : False,
    'add_cross': True,
    'add_tanh': False
}

import time

start = time.time()


x_split, y_split, jet_num_to_idx = pp.preprocess_jet_num(x, y, to_replace, do_normalise=True, augment_param=augment_param)

end = time.time()
print(end - start)

0 (99913, 30) (250000,)
1 (77544, 30) (250000,)
2 (72543, 30) (250000,)
52.0356388092041


In [6]:
x_split[2].shape, y_split[2].shape

((72543, 464), (72543,))

In [7]:
data_part = []

nb_samples = []
for x_, y_ in zip(x_split, y_split):
    train_data, test_data = misc.random_data_split(y_, x_, k_fold=10)
    data = {'train': train_data, 'test': test_data}
    data_part.append(data)
    nb_samples.append(len(test_data[1]))

In [8]:
models = []
learning_param = { 'lambda_': 1e-5 }

acc_list = []

for idx in range(len(data_part)):
    x_tr, y_tr = data_part[idx]['train']
    nb_features = x_tr.shape[1]
    
    w_initial = np.random.uniform(0,1,size=nb_features) 
    model_ls = model.Model('ridge_regression', w_initial, learning_param, debug=True)
    
    model_ls.learn_weights(y_tr, x_tr)
    models.append(model_ls)
    
    tx_te, y_te = data_part[idx]['test']
    y_ = model_ls.predict(tx_te)
    acc_list.append(misc.accuracy(y_te, y_))

learning weights...
done.
learning weights...
done.
learning weights...
done.


In [9]:
acc_list

[0.8330497447702933, 0.7788238328604591, 0.8001102839812517]

In [10]:
avg_acc = np.dot(acc_list, nb_samples) / sum(nb_samples)
avg_acc

0.8066722668906756

In [11]:
# Save weights
weights = [trained_model.w for trained_model in models]
file_name = 'weights.npy'
np.save(file_name, weights) # save

In [12]:
weights[2].shape

(464,)

In [13]:
weights_from_file = np.load(file_name) # load

In [14]:
weights_from_file[0].shape

(209,)